In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import polars

In [3]:
data = polars.read_csv('/mnt/processed/private/msds-pt2025b/lt4/final_project/train_base.csv')

In [5]:
data.shape

(126651, 472)

In [6]:
data.head()

case_id,month,week_num,target,assignmentdate_238D,assignmentdate_4527235D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtcount_4527229L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,…,mean_mainoccupationinc_384A,max_amount_416A,max_num_group1_10,max_openingdate_313D,mean_amount_416A,mean_openingdate_313D,max_num_group1_11,max_openingdate_857D,mean_openingdate_857D,max_collater_typofvalofguarant_298M,max_collater_typofvalofguarant_407M,max_collater_valueofguarantee_1124L,max_collater_valueofguarantee_876L,max_collaterals_typeofguarante_359M,max_collaterals_typeofguarante_669M,max_num_group1_12,max_num_group2,max_pmts_dpd_1073P,max_pmts_dpd_303P,max_pmts_month_158T,max_pmts_month_706T,max_pmts_overdue_1140A,max_pmts_overdue_1152A,max_pmts_year_1139T,max_pmts_year_507T,max_subjectroles_name_541M,max_subjectroles_name_838M,mean_pmts_dpd_1073P,mean_pmts_dpd_303P,mean_pmts_overdue_1140A,mean_pmts_overdue_1152A,var_pmts_dpd_1073P,var_pmts_dpd_303P,var_pmts_overdue_1140A,var_pmts_overdue_1152A,year,day
i64,i64,i64,i64,i64,i64,i64,f64,i64,f64,f64,f64,f64,f64,str,str,str,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,…,f64,f64,i64,i64,f64,i64,i64,i64,i64,str,str,f64,f64,str,str,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
1402704,201906,23,0,null,null,-12127,null,-12127,1.0,1.0,0.0,1.0,0.0,"""a55475b1""","""a55475b1""","""a55475b1""",1.0,0.0,"""a55475b1""","""a55475b1""",1.0,null,null,null,null,5.0,74996.805,null,14,null,null,1.0,0.0,0.0,83502.14,3833.4001,…,80000.0,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,0.0,"""c7a5ad39""","""a55475b1""",0,23,null,0.0,null,12.0,null,0.0,null,2020.0,"""ab3c25cf""","""a55475b1""",null,0.0,null,0.0,null,0.0,null,0.0,2019,14
1482183,201908,31,0,-2705,null,-23902,null,-23902,1.0,1.0,1.0,1.0,1.0,"""a55475b1""","""a55475b1""","""a55475b1""",0.0,0.0,"""b6cabe76""","""a55475b1""",1.0,16674.201,null,null,6.0,null,null,null,14,null,null,0.0,1.0,0.0,0.0,4809.6,…,22400.0,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,200000.0,"""c7a5ad39""","""a55475b1""",1,23,null,0.0,null,12.0,null,0.0,null,2015.0,"""ab3c25cf""","""a55475b1""",null,0.0,null,0.0,null,0.0,null,0.0,2019,9
856351,201911,44,0,null,null,null,null,-13128,0.0,0.0,0.0,1.0,0.0,"""a55475b1""","""a55475b1""","""a55475b1""",0.0,1.0,"""a55475b1""","""a55475b1""",1.0,null,null,null,null,null,null,"""DEDUCTION_6""",null,14,null,1.0,0.0,null,null,7727.2,…,80000.0,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,0.0,"""c7a5ad39""","""a55475b1""",9,35,null,11.0,null,12.0,null,1393.248,null,2019.0,"""ab3c25cf""","""a55475b1""",null,0.111111,null,12.990037,null,1.1277258,null,17972.053,2019,10
1927558,202009,88,0,null,null,null,null,-20620,0.0,0.0,0.0,0.0,0.0,"""2fc785b2""","""a55475b1""","""a55475b1""",1.0,1.0,"""a55475b1""","""a55475b1""",0.0,null,null,null,null,null,null,null,null,null,14,0.0,1.0,0.0,17261.201,7436.0,…,90000.0,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,0.0,"""c7a5ad39""","""a55475b1""",2,23,null,0.0,null,12.0,null,0.0,null,2020.0,"""ab3c25cf""","""a55475b1""",null,0.0,null,0.0,null,0.0,null,0.0,2020,14
2676673,202003,62,0,null,null,null,null,-16874,0.0,2.0,0.0,2.0,0.0,"""a55475b1""","""a55475b1""","""a55475b1""",0.0,4.0,"""a55475b1""","""a55475b1""",2.0,null,null,null,null,null,null,"""DEDUCTION_6""",null,14,null,0.0,0.0,0.0,164638.36,7338.8003,…,62000.0,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",0.0,0.0,

In [7]:
data = data.to_pandas()

In [8]:
data.shape

(126651, 472)

In [12]:
cat_cols = [col for col, dtype in dict(data.dtypes).items() if dtype == 'O']
data[cat_cols] = data[cat_cols].astype('category')

In [13]:
from sklearn.model_selection import train_test_split


X = data.drop(['month', 'week_num', 'target'], axis = 1)
y = data['target']
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
score_schema = {'model_name': 'str', 
                        'model': 'str',
                        'params': 'str',
                        'accuracy': 'float',
                        'precision': 'float',
                        'recall': 'float',
                        'f1_score': 'float',
                        'roc_auc': 'float',
                        'pr_auc': 'float',
                        'lift': 'float',
                        'accuracy_cv': 'float',
                        'precision_cv': 'float',
                        'recall_cv': 'float',
                        'f1_score_cv': 'float',
                        'roc_auc_cv': 'float',
                        'pr_auc_cv': 'float',
                        'lift_cv': 'float'}
scores = pd.DataFrame(columns = score_schema.keys()).astype(score_schema)
scores

,model_name,model,params,accuracy,precision,recall,f1_score,roc_auc,pr_auc,lift,accuracy_cv,precision_cv,recall_cv,f1_score_cv,roc_auc_cv,pr_auc_cv,lift_cv


In [19]:
feature_cols = X.columns[1:]

In [20]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from utils.model_helpers import model_evals
import xgboost
import lightgbm
from time import time
from sklearn.model_selection import cross_validate, StratifiedKFold
import numpy as np
from utils.model_helpers import cutoff_perc, model_evals

space = {
    'max_depth': hp.quniform("max_depth", 3, 10, 1),  #Max depth 10
    'gamma': hp.loguniform('gamma', np.log(1), np.log(100)),
    'reg_alpha': hp.uniform('reg_alpha', 0, 10),
    'reg_lambda': hp.uniform('reg_lambda', 0, 10),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight',np.log(1), np.log(100)),
    'n_estimators': hp.quniform('n_estimators', 50, 250, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'random_state': 42,   
    'verbose': -1
}

def objective(space):
    start_time = time()

    print(f"Parameters: {space}")
    space['max_depth'] = int(space['max_depth'])
    #space['reg_alpha'] = int(space['reg_alpha'])
    space['n_estimators'] = int(space['n_estimators'])

    cv_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    i = 1
    
    model_name = f'LGB_Tune{len(scores)}'
    model_type = 'LightGBM'
    params = 'default'
    dict_results = {'acc': [],
                        'precision': [],
                        'recall': [],
                        'f1': [],
                        'roc_auc': [],
                        'pr_auc': [],
                        'lift': []}
    
    for train_index, test_index in cv_folds.split(X_train_val[feature_cols], y_train_val):

        print(f"CV Fold: {i}")
        # Get the training and testing data for this fold
        X_train_cv, X_test_cv = X_train_val.iloc[train_index], X_train_val.iloc[test_index]
        y_train_cv, y_test_cv = y_train_val.iloc[train_index], y_train_val.iloc[test_index]

        lgb = lightgbm.LGBMClassifier(**space)
        lgb.fit(X_train_cv, y_train_cv)

        y_test_proba_cv = lgb.predict_proba(X_test_cv)[:, 1]
        results = model_evals(y_test_cv, y_test_proba_cv)
        print(f'LightGBM, Val CV{i} Tuned: {results}" ')

        i += 1
        
        for key in dict_results.keys():
            dict_results[key].append(results[key])
    
    lgb.fit(X_train_val, y_train_val)
    y_test_proba = lgb.predict_proba(X_test)[:, 1]
    test_results = model_evals(y_test, y_test_proba)
    print(f'LigthGBM, OOT Tuned: {test_results}" ')

    elapsed_time = time() - start_time
    
    row = [model_name, model_type, lgb]
    for key in dict_results.keys():
        row.append(test_results[key])

    for key in dict_results.keys():
        row.append(np.array(dict_results[key]).mean())

    
    print(row)
    scores.loc[len(scores)] = row
    scores.to_csv('../data/LGB_Tuned_Mobile.csv')

    return {'loss': -test_results['pr_auc'], 'status': STATUS_OK, 'model': lgb, 'elapsed_time': elapsed_time}


In [ ]:
trials = Trials()
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 250, trials = trials)

Parameters: {'colsample_bytree': 0.789453478290231, 'gamma': 37.824530939163736, 'learning_rate': 0.01565456848540161, 'max_depth': 6.0, 'min_child_weight': 11.39398330882743, 'n_estimators': 80.0, 'random_state': 42, 'reg_alpha': 9.765784982987437, 'reg_lambda': 8.936204136622948, 'subsample': 0.8066223904381988, 'verbose': -1}
CV Fold: 1                                             
LightGBM, Val CV1 Tuned: {'event_rate': 0.0, 'acc': 0.9686142913541256, 'roc_auc': 0.7900861687769878, 'pr_auc': 0.12086250172586943, 'recall': 0.0, 'precision': 0.0, 'f1': 0.0, 'lift': nan}" 
CV Fold: 2                                             
  0%|          | 0/250 [00:07<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

/home/msds-pt2025b/jco/term2_acs/final_project/HOME_CREDIT_KAGGLE/notebooks/utils/model_helpers.py:45: RuntimeWarning: invalid value encountered in scalar divide
  lift = recall / event_rate



LightGBM, Val CV2 Tuned: {'event_rate': 0.0, 'acc': 0.9686142913541256, 'roc_auc': 0.8100955284005779, 'pr_auc': 0.14818536154082015, 'recall': 0.0, 'precision': 0.0, 'f1': 0.0, 'lift': nan}" 
CV Fold: 3                                             
  0%|          | 0/250 [00:14<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

/home/msds-pt2025b/jco/term2_acs/final_project/HOME_CREDIT_KAGGLE/notebooks/utils/model_helpers.py:45: RuntimeWarning: invalid value encountered in scalar divide
  lift = recall / event_rate



LightGBM, Val CV3 Tuned: {'event_rate': 0.0, 'acc': 0.9686142913541256, 'roc_auc': 0.8052062786059703, 'pr_auc': 0.1273384899770608, 'recall': 0.0, 'precision': 0.0, 'f1': 0.0, 'lift': nan}" 
CV Fold: 4                                             
  0%|          | 0/250 [00:21<?, ?trial/s, best loss=?]

...

LigthGBM, OOT Tuned: {'event_rate': 0.0, 'acc': 0.9675101654099719, 'roc_auc': 0.8117565102852324, 'pr_auc': 0.15108209142665607, 'recall': 0.0, 'precision': 0.0, 'f1': 0.0, 'lift': nan}" 
['LGB_Tune55', 'LightGBM', LGBMClassifier(colsample_bytree=0.7314618949770157, gamma=2.799062335982357,
               learning_rate=0.025866055243263568, max_depth=4,
               min_child_weight=62.37745252582922, n_estimators=115,
               random_state=42, reg_alpha=1.8370587491801227,
               reg_lambda=3.0302168086924444, subsample=0.651543995767057,
               verbose=-1), 0.9675101654099719, 0.0, 0.0, 0.0, 0.8117565102852324, 0.15108209142665607, nan, 0.9685945519147257, 0.0, 0.0, 0.0, 0.816839056432465, 0.1533309750644983, nan]
Parameters: {'colsample_bytree': 0.9967462801645117, 'gamma': 8.852874001856815, 'learning_rate': 0.15494976401582597, 'max_depth': 5.0, 'min_child_weight': 1.685225539119468, 'n_estimators': 123.0, 'random_state': 42, 'reg_alpha': 9.221007564228932

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

/home/msds-pt2025b/jco/term2_acs/final_project/HOME_CREDIT_KAGGLE/notebooks/utils/model_helpers.py:45: RuntimeWarning: invalid value encountered in scalar divide
  lift = recall / event_rate



CV Fold: 1                                                                              
LightGBM, Val CV1 Tuned: {'event_rate': 0.0009376233714962495, 'acc': 0.9691571259376234, 'roc_auc': 0.8362704319205139, 'pr_auc': 0.17118924679472658, 'recall': 0.02358490566037736, 'precision': 0.7894736842105263, 'f1': 0.04580152671755726, 'lift': 25.15392254220457}" 
CV Fold: 2                                                                              
LightGBM, Val CV2 Tuned: {'event_rate': 0.0009376233714962495, 'acc': 0.9687623371496249, 'roc_auc': 0.842800139192759, 'pr_auc': 0.1848172075091733, 'recall': 0.01729559748427673, 'precision': 0.5789473684210527, 'f1': 0.03358778625954198, 'lift': 18.44620986428335}" 
CV Fold: 3                                                                              
LightGBM, Val CV3 Tuned: {'event_rate': 0.0013324121594946703, 'acc': 0.9682688511646269, 'roc_auc': 0.8343870519973392, 'pr_auc': 0.16384545715923182, 'recall': 0.015723270440251572, 'precis

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

/home/msds-pt2025b/jco/term2_acs/final_project/HOME_CREDIT_KAGGLE/notebooks/utils/model_helpers.py:45: RuntimeWarning: invalid value encountered in scalar divide
  lift = recall / event_rate



CV Fold: 1                                                                              
LightGBM, Val CV1 Tuned: {'event_rate': 0.0013324121594946703, 'acc': 0.9690584287406238, 'roc_auc': 0.8334790467474907, 'pr_auc': 0.175855922818301, 'recall': 0.02830188679245283, 'precision': 0.6666666666666666, 'f1': 0.05429864253393665, 'lift': 21.241090146750526}" 
CV Fold: 2                                                                              
LightGBM, Val CV2 Tuned: {'event_rate': 0.0013324121594946703, 'acc': 0.9690584287406238, 'roc_auc': 0.8416500526138375, 'pr_auc': 0.18614195806926762, 'recall': 0.02830188679245283, 'precision': 0.6666666666666666, 'f1': 0.05429864253393665, 'lift': 21.241090146750526}" 
CV Fold: 3                                                                              
LightGBM, Val CV3 Tuned: {'event_rate': 0.0015791551519936833, 'acc': 0.9684168969601263, 'roc_auc': 0.8390417104047229, 'pr_auc': 0.17172057562950313, 'recall': 0.0220125786163522, 'precis